In [1]:
from datetime import timedelta, datetime
import glob
import json
import os
import re

from collections import Counter

import numpy as np
import pandas as pd

import scipy.sparse as spr
import pickle

In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive/')

In [3]:
# 태그는 이걸로
kihong = pd.read_json('results_best_0724tag.json', typ = 'frame',encoding='UTF-8')
# song은 이걸로
hyn = pd.read_json('adds_base50.json', typ = 'frame',encoding='UTF-8')

In [4]:
hyn.head(3)

,id,songs,tags
0,118598,"[207912, 623047, 703323, 422438, 638488, 32221...","[OST, 디즈니, 애니메이션, 영화, 추억, 픽사, 기분전환, 인어공주, 겨울왕국..."
1,131447,"[366786, 675367, 678294, 660937, 334372, 27799...","[락, 기분전환, 회상, 감성, 추억, 잔잔한, 클럽, 신나는, 힐링, 휴식]"
2,51464,"[411161, 230907, 325455, 253755, 600676, 39636...","[발라드, 추억, 회상, 잔잔한, 설렘, 사랑, 밤, 새벽, 이별, 슬픔]"


In [5]:
val = pd.read_json('val.json', typ = 'frame',encoding='UTF-8')

In [6]:
val.shape

(23015, 6)

In [7]:
# song & song 모두 있는 id 추출
val['len_songs'] = val['songs'].apply(len)
val['len_tags'] = val['tags'].apply(len)

In [8]:
val['len_songs'].describe()

count    23015.000000
mean        18.301065
std         21.544313
min          0.000000
25%          6.000000
50%         11.000000
75%         23.000000
max        100.000000
Name: len_songs, dtype: float64

In [9]:
len(val)

23015

In [10]:
hyn['len_songs'] = hyn['songs'].apply(len)

In [11]:
# 100개 아닌거 생김
less = hyn.query('len_songs!=100')
less_id = less['id']

In [12]:
less_id

Series([], Name: id, dtype: int64)

In [13]:
# val_kihong = pd.merge(val, kihong, on='id')

In [14]:
a = pd.merge(kihong, hyn, on='id') # x가 기홍

In [15]:
# strange = val_kihong.query('len_songs==0 | len_songs==1')
strange = a.query('id==@less_id')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3346: PerformanceWarning: Alignment difference on axis 0 is larger than an order of magnitude on term '__pd_eval_local_less_id', by more than 4.362; performance may suffer
  return _eval(expr, inplace=inplace, **kwargs)


In [16]:
strange

,id,songs_x,tags_x,songs_y,tags_y,len_songs


In [17]:
len(strange) # y(태현)껄로 예측해야함

0

In [18]:
strange_id = strange['id'].tolist()

In [19]:
strange['new_song'] = strange['songs_x']

In [20]:
strange = strange.reset_index()

In [21]:
strange.head(1)

,index,id,songs_x,tags_x,songs_y,tags_y,len_songs,new_song


In [22]:
not_strange = a.query('id!=@less_id')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3346: PerformanceWarning: Alignment difference on axis 0 is larger than an order of magnitude on term '__pd_eval_local_less_id', by more than 4.362; performance may suffer
  return _eval(expr, inplace=inplace, **kwargs)


In [23]:
not_strange['new_song'] = not_strange['songs_y'] # 안이상하면 태현코드

In [24]:
not_strange.head(1)

,id,songs_x,tags_x,songs_y,tags_y,len_songs,new_song
0,118598,"[207912, 623047, 703323, 422438, 638488, 32221...","[OST, 디즈니, 애니메이션, 영화, 디즈니OST, 추억, 기분전환, 겨울왕국, ...","[207912, 623047, 703323, 422438, 638488, 32221...","[OST, 디즈니, 애니메이션, 영화, 추억, 픽사, 기분전환, 인어공주, 겨울왕국...",100,"[207912, 623047, 703323, 422438, 638488, 32221..."


In [25]:
not_strange = not_strange.reset_index()

In [26]:
not_strange.head(1)

,index,id,songs_x,tags_x,songs_y,tags_y,len_songs,new_song
0,0,118598,"[207912, 623047, 703323, 422438, 638488, 32221...","[OST, 디즈니, 애니메이션, 영화, 디즈니OST, 추억, 기분전환, 겨울왕국, ...","[207912, 623047, 703323, 422438, 638488, 32221...","[OST, 디즈니, 애니메이션, 영화, 추억, 픽사, 기분전환, 인어공주, 겨울왕국...",100,"[207912, 623047, 703323, 422438, 638488, 32221..."


In [27]:
results = pd.concat([strange, not_strange])

In [28]:
results = results.sort_values(by='index')

In [29]:
results = results.reset_index()

In [30]:
results

,level_0,index,id,songs_x,tags_x,songs_y,tags_y,len_songs,new_song
0,0,0,118598,"[207912, 623047, 703323, 422438, 638488, 32221...","[OST, 디즈니, 애니메이션, 영화, 디즈니OST, 추억, 기분전환, 겨울왕국, ...","[207912, 623047, 703323, 422438, 638488, 32221...","[OST, 디즈니, 애니메이션, 영화, 추억, 픽사, 기분전환, 인어공주, 겨울왕국...",100,"[207912, 623047, 703323, 422438, 638488, 32221..."
1,1,1,131447,"[238309, 502217, 334372, 675367, 16061, 621677...","[OST, 디즈니, 애니메이션, 영화, 기분전환, 추억, 휴식, 힐링, 디즈니OST...","[366786, 675367, 678294, 660937, 334372, 27799...","[락, 기분전환, 회상, 감성, 추억, 잔잔한, 클럽, 신나는, 힐링, 휴식]",100,"[366786, 675367, 678294, 660937, 334372, 27799..."
2,2,2,51464,"[291080, 659416, 425043, 500248, 358818, 29532...","[발라드, 이별, 슬픔, 추억, 회상, 밤, 새벽, 잔잔한, 설렘, 사랑]","[411161, 230907, 325455, 253755, 600676, 39636...","[발라드, 추억, 회상, 잔잔한, 설렘, 사랑, 밤, 새벽, 이별, 슬픔]",100,"[411161, 230907, 325455, 253755, 600676, 39636..."
3,3,3,45144,"[144663, 367963, 351888, 19533, 174749, 357367...","[감성, 사랑, 이별, 발라드, 잔잔한, 새벽, 기분전환, 가을, 카페, 인디]","[144663, 367963, 174749, 224921, 362966, 44924...","[감성, 사랑, 발라드, 밤, 비, 이별, 잔잔한, 새벽, 힐링, 카페]",100,"[144663, 367963, 174749, 224921, 362966, 44924..."
4,4,4,79929,"[211220, 412769, 70314, 106129, 526037, 242124...","[CCM, 찬양, 예배, 은혜, 워십, 국내ccm, 교회, 찬송, 제이어스, 감사]","[211220, 445299, 70314, 412769, 618001, 106129...","[CCM, 찬양, 은혜, 국내ccm, 사랑, 파워DJ_CCM, 은혜로운, 위로, 찬...",100,"[211220, 445299, 70314, 412769, 618001, 106129..."
...,...,...,...,...,...,...,...,...,...
23010,23010,23010,101722,"[116573, 366786, 51612, 339802, 276418, 13142,...","[새벽, 밤, 이별, 추억, 슬픔, 회상, 힐링, 휴식, 발라드, 감성]","[116573, 339004, 104677, 219420, 532347, 55358...","[로맨틱쇼, 센언니, 오래보자, 레오, 차트정복, 관능, 평생소장, 저녁드라이브, ...",100,"[116573, 339004, 104677, 219420, 532347, 55358..."
23011,23011,23011,122127,"[352228, 48209, 138932, 630552, 473514, 4173, ...","[추억, 회상, 사랑, 힐링, 엄마, 휴식, 부모님, 잔잔한, 발라드, 아버지]","[352228, 4173, 138932, 630552, 473514, 676492,...","[엄마, 부모님, 아버지, 사랑, 눈물, 어버이날, 위로, 감사, 감성, 힐링]",100,"[352228, 4173, 138932, 630552, 473514, 676492,..."
23012,23012,23012,77438,"[274504, 140867, 679436, 493762, 360062, 21125...","[팝, 팝송, Pop, 기분전환, 드라이브, 카페, 휴식, 힐링, 감성, 잔잔한]","[274504, 679436, 459256, 542127, 140867, 49376...","[Pop, 팝송, 팝, 기분전환, 드라이브, 해외차트, 시대별차트, 좋아요, 휴식,...",100,"[274504, 679436, 459256, 542127, 140867, 49376..."
23013,23013,23013,36231,"[548041, 50031, 46497, 699175, 236711, 198144,...","[클래식, 피아노, 힐링, 잔잔한, 조성진, 휴식, 기분전환, 쇼팽, classic...","[354737, 64680, 375318, 532729, 235651, 81796,...","[클래식, 힐링, 휴식, 잔잔한, 팝, 연주곡, 독서, 추억, 회상, 감성]",100,"[354737, 64680, 375318, 532729, 235651, 81796,..."


In [31]:
results = results.rename(columns={'new_song':'songs', 'tags_x':'tags'})

In [32]:
results.drop(['songs_x', 'songs_y', 'tags_y', 'index', 'level_0'], inplace=True, axis=1)

In [33]:
results = results[['id', 'songs', 'tags']]

In [34]:
results.head()

,id,songs,tags
0,118598,"[207912, 623047, 703323, 422438, 638488, 32221...","[OST, 디즈니, 애니메이션, 영화, 디즈니OST, 추억, 기분전환, 겨울왕국, ..."
1,131447,"[366786, 675367, 678294, 660937, 334372, 27799...","[OST, 디즈니, 애니메이션, 영화, 기분전환, 추억, 휴식, 힐링, 디즈니OST..."
2,51464,"[411161, 230907, 325455, 253755, 600676, 39636...","[발라드, 이별, 슬픔, 추억, 회상, 밤, 새벽, 잔잔한, 설렘, 사랑]"
3,45144,"[144663, 367963, 174749, 224921, 362966, 44924...","[감성, 사랑, 이별, 발라드, 잔잔한, 새벽, 기분전환, 가을, 카페, 인디]"
4,79929,"[211220, 445299, 70314, 412769, 618001, 106129...","[CCM, 찬양, 예배, 은혜, 워십, 국내ccm, 교회, 찬송, 제이어스, 감사]"


In [35]:
res = []
for i in results.index :
  res.append({
      "id" : results.iloc[i, 0],
      "songs" : results.iloc[i, 1],
      "tags":results.iloc[i, 2]
  })


In [36]:
import io
import os

def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    with io.open(fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

write_json(res, 'results_k50ing.json')

In [37]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23015 entries, 0 to 23014
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      23015 non-null  int64 
 1   songs   23015 non-null  object
 2   tags    23015 non-null  object
dtypes: int64(1), object(2)
memory usage: 539.5+ KB


In [38]:
kihong.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23015 entries, 0 to 23014
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      23015 non-null  int64 
 1   songs   23015 non-null  object
 2   tags    23015 non-null  object
dtypes: int64(1), object(2)
memory usage: 539.5+ KB
